In [28]:
import torch
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as T
import torch.nn.functional as F
import torch.nn as nn

In [29]:
epochs = 10
bs = 256

from typing import Tuple
from torch import Tensor
from torch.utils.data import Dataset

class TensorDataset(Dataset[Tuple[Tensor, ...]]):
    """Dataset wrapping tensors.
    Each sample will be retrieved by indexing tensors along the first dimension.
    Args:
        *tensors (Tensor): tensors that have the same size of the first dimension.
    """
    tensors: Tuple[Tensor, ...]

    def __init__(self, *tensors: Tensor) -> None:
        assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors), "Size mismatch between tensors"
        self.tensors = tensors

    def __getitem__(self, index):
        return (*tuple(tensor[index] for tensor in self.tensors), index)

    def __len__(self):
        return self.tensors[0].size(0)

In [34]:
mtrain = torchvision.datasets.MNIST("../data/", train=True, download=True, transform=T.Compose([T.ToTensor(),T.Normalize((0.1307,), (0.3081,)), T.Lambda(lambda x: torch.flatten(x))]))
mtest = torchvision.datasets.MNIST("../data/", train=False, download=True, transform=T.Compose([T.ToTensor(), T.Normalize((0.1307,), (0.3081,)), T.Lambda(lambda x: torch.flatten(x))]))
mtrain_loader = DataLoader(mtrain, batch_size=bs, shuffle=True)
mtest_loader = DataLoader(mtest, batch_size=bs, shuffle=True)

ktrain = torchvision.datasets.KMNIST("../data/", train=True, download=True, transform=T.Compose([T.ToTensor(),T.Normalize((0.5,), (0.5,)), T.Lambda(lambda x: torch.flatten(x))]))
ktest = torchvision.datasets.KMNIST("../data/", train=False, download=True, transform=T.Compose([T.ToTensor(), T.Normalize((0.5,), (0.5,)), T.Lambda(lambda x: torch.flatten(x))]))
ktrain_loader = DataLoader(ktrain, batch_size=bs, shuffle=True)
ktest_loader = DataLoader(ktest, batch_size=bs, shuffle=True)

ftrain = torchvision.datasets.FashionMNIST("../data/", train=True, download=True, transform=T.Compose([T.ToTensor(),T.Normalize((0.1307,), (0.3081,)), T.Lambda(lambda x: torch.flatten(x))]))
ftest = torchvision.datasets.FashionMNIST("../data/", train=False, download=True, transform=T.Compose([T.ToTensor(), T.Normalize((0.1307,), (0.3081,)), T.Lambda(lambda x: torch.flatten(x))]))
ftrain_loader = DataLoader(ftrain, batch_size=bs, shuffle=True)
ftest_loader = DataLoader(ftest, batch_size=bs, shuffle=True)

def accuracy(model, loader):
    device = next(model.parameters()).device
    correct = 0
    total = 0
    for inputs, _, targets,_ in loader:
        outputs = model(inputs.to(device).float())
        correct += torch.sum(torch.argmax(outputs, dim=1) == targets.to(device))
        total += inputs.shape[0]
    return correct / total * 100

Files already downloaded and verified
Files already downloaded and verified


In [35]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 100)
        self.fc2 = nn.Linear(100, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

In [36]:
for epoch in tqdm(range(epochs)):
    for inputs, _, targets, _ in mtrain_loader:
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

100%|██████████| 10/10 [00:04<00:00,  2.29it/s]
